In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import ttest_ind, f_oneway
df = pd.read_csv('../../data/log_data_fe.csv')
df

,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,LAG_3_HOLIDAY_FLAG,LEAD_3_HOLIDAY_FLAG,SEASON,MONTH,CHANNEL_ATL_OR_DR,CHANNEL_CAMPAIGN_TYPE,COST_PER_FREE_TRIALS,COST_BY_FREE_TRIALS,LOG_COST,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,False,False,Summer,June,paid social_DR - Direct Response,paid social_Title,0.090095,6.725733e+08,8.959994,11.366768,0.086265,20.326622
1,DR - Direct Response,Title,app,2474.31,2956.74,False,False,False,Summer,August,app_DR - Direct Response,app_Title,0.836837,7.315891e+06,7.814121,7.992181,0.608045,15.805560
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,False,False,Summer,July,paid social_ATL - Above The Line,paid social_Title,0.816971,1.279190e+08,9.232476,9.434610,0.597171,18.666908
3,DR - Direct Response,Title,app,49631.87,17207.04,False,True,False,Winter,December,app_DR - Direct Response,app_Title,2.884393,8.540176e+08,10.812409,9.753132,1.356967,20.565462
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,False,False,Summer,August,paid social_ATL - Above The Line,paid social_Title,0.095644,4.528039e+07,7.641113,9.987798,0.091343,17.628385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14103,ATL - Above The Line,Brand,bvod,1478.78,16941.67,False,False,False,Winter,February,bvod_ATL - Above The Line,bvod_Brand,0.087287,2.505300e+07,7.299649,9.737591,0.083685,17.036504
14104,ATL - Above The Line,Title,ooh,839.25,5727.03,False,False,False,Spring,April,ooh_ATL - Above The Line,ooh_Title,0.146542,4.806410e+06,6.733699,8.653127,0.136750,15.385461
14105,ATL - Above The Line,Title,bvod,193796.71,34824.51,False,False,False,Winter,December,bvod_ATL - Above The Line,bvod_Title,5.564952,6.748875e+09,12.174570,10.458105,1.881745,22.632642
14106,ATL - Above The Line,Brand,paid social,84690.65,25140.77,False,False,False,Winter,February,paid social_ATL - Above The Line,paid social_Brand,3.368658,2.129188e+09,11.346772,10.132286,1.474456,21.479007


In [11]:
binary_mappings = {
    'binary': ['ATL_OR_DR', 'HOLIDAY_FLAG', 'LAG_3_HOLIDAY_FLAG', 'LEAD_3_HOLIDAY_FLAG'],
    'nonbinary': ['CAMPAIGN_TYPE', 'CHANNEL', 'SEASON', 'CHANNEL_ATL_OR_DR', 'CHANNEL_CAMPAIGN_TYPE', 'MONTH']
}

for dataType, columnName in binary_mappings.items():
    if dataType == 'binary':
        for cn in columnName:
            group1 = df[df[cn ] == df[cn].unique()[0]]['LOG_FREE_TRIALS']
            group2 = df[df[cn ] == df[cn].unique()[1]]['LOG_FREE_TRIALS']

            # Perform Independent T-Test
            t_stat, p_value = ttest_ind(group1, group2)

            # Print result dynamically
            print(f'{cn}:\n tStat: {t_stat:.2f} | pValue: {p_value:.2f}')
    else:
        for cn in columnName:
            groups = [df[df[cn] == val]['LOG_FREE_TRIALS'] for val in df[cn].unique()]

            # Perform One-Way ANOVA
            f_stat, p_value = f_oneway(*groups)

            print(f'{cn}:\n fStat: {t_stat:.2f} | pValue: {p_value:.2f}')

ATL_OR_DR:
 tStat: -0.01 | pValue: 0.99
HOLIDAY_FLAG:
 tStat: 0.62 | pValue: 0.53
LAG_3_HOLIDAY_FLAG:
 tStat: -0.88 | pValue: 0.38
LEAD_3_HOLIDAY_FLAG:
 tStat: 0.74 | pValue: 0.46
CAMPAIGN_TYPE:
 fStat: 0.74 | pValue: 0.66
CHANNEL:
 fStat: 0.74 | pValue: 0.79
SEASON:
 fStat: 0.74 | pValue: 0.49
CHANNEL_ATL_OR_DR:
 fStat: 0.74 | pValue: 0.85
CHANNEL_CAMPAIGN_TYPE:
 fStat: 0.74 | pValue: 0.63
MONTH:
 fStat: 0.74 | pValue: 0.20


In [12]:
'''
1. We can confirm that holiday doesn't show significant difference in the impact of cost.
'''
for dataType, columnName in binary_mappings.items():
    if dataType == 'binary':
        for cn in columnName:
            group1 = df[df[cn ] == df[cn].unique()[0]]['LOG_COST']
            group2 = df[df[cn ] == df[cn].unique()[1]]['LOG_COST']

            # Perform Independent T-Test
            t_stat, p_value = ttest_ind(group1, group2)

            # Print result dynamically
            print(f'{cn}:\n tStat: {t_stat:.2f} | pValue: {p_value:.2f}')
    else:
        for cn in columnName:
            groups = [df[df[cn] == val]['LOG_COST'] for val in df[cn].unique()]

            # Perform One-Way ANOVA
            f_stat, p_value = f_oneway(*groups)

            print(f'{cn}:\n fStat: {t_stat:.2f} | pValue: {p_value:.2f}')

ATL_OR_DR:
 tStat: -1.29 | pValue: 0.20
HOLIDAY_FLAG:
 tStat: -1.50 | pValue: 0.13
LAG_3_HOLIDAY_FLAG:
 tStat: 0.97 | pValue: 0.33
LEAD_3_HOLIDAY_FLAG:
 tStat: -0.89 | pValue: 0.37
CAMPAIGN_TYPE:
 fStat: -0.89 | pValue: 0.63
CHANNEL:
 fStat: -0.89 | pValue: 0.63
SEASON:
 fStat: -0.89 | pValue: 0.75
CHANNEL_ATL_OR_DR:
 fStat: -0.89 | pValue: 0.73
CHANNEL_CAMPAIGN_TYPE:
 fStat: -0.89 | pValue: 0.16
MONTH:
 fStat: -0.89 | pValue: 0.44


In [13]:
df.drop(columns=['FREE_TRIALS'], axis=1, inplace=True)
df

,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,HOLIDAY_FLAG,LAG_3_HOLIDAY_FLAG,LEAD_3_HOLIDAY_FLAG,SEASON,MONTH,CHANNEL_ATL_OR_DR,CHANNEL_CAMPAIGN_TYPE,COST_PER_FREE_TRIALS,COST_BY_FREE_TRIALS,LOG_COST,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS
0,DR - Direct Response,Title,paid social,7784.31,False,False,False,Summer,June,paid social_DR - Direct Response,paid social_Title,0.090095,6.725733e+08,8.959994,11.366768,0.086265,20.326622
1,DR - Direct Response,Title,app,2474.31,False,False,False,Summer,August,app_DR - Direct Response,app_Title,0.836837,7.315891e+06,7.814121,7.992181,0.608045,15.805560
2,ATL - Above The Line,Title,paid social,10222.82,False,False,False,Summer,July,paid social_ATL - Above The Line,paid social_Title,0.816971,1.279190e+08,9.232476,9.434610,0.597171,18.666908
3,DR - Direct Response,Title,app,49631.87,False,True,False,Winter,December,app_DR - Direct Response,app_Title,2.884393,8.540176e+08,10.812409,9.753132,1.356967,20.565462
4,ATL - Above The Line,Title,paid social,2081.06,False,False,False,Summer,August,paid social_ATL - Above The Line,paid social_Title,0.095644,4.528039e+07,7.641113,9.987798,0.091343,17.628385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14103,ATL - Above The Line,Brand,bvod,1478.78,False,False,False,Winter,February,bvod_ATL - Above The Line,bvod_Brand,0.087287,2.505300e+07,7.299649,9.737591,0.083685,17.036504
14104,ATL - Above The Line,Title,ooh,839.25,False,False,False,Spring,April,ooh_ATL - Above The Line,ooh_Title,0.146542,4.806410e+06,6.733699,8.653127,0.136750,15.385461
14105,ATL - Above The Line,Title,bvod,193796.71,False,False,False,Winter,December,bvod_ATL - Above The Line,bvod_Title,5.564952,6.748875e+09,12.174570,10.458105,1.881745,22.632642
14106,ATL - Above The Line,Brand,paid social,84690.65,False,False,False,Winter,February,paid social_ATL - Above The Line,paid social_Brand,3.368658,2.129188e+09,11.346772,10.132286,1.474456,21.479007


In [ ]:
# Step 1: Get unique categories
categories = df['Category'].unique()
print("Unique categories:", categories)

# Step 2: Group data by category
grouped_data = {}
for category in categories:
    grouped_data[category] = df[df['Category'] == category]['Free_Trials'].values
print("Grouped data:", grouped_data)

# Step 3: Convert grouped data to a list of arrays
grouped_data_list = list(grouped_data.values())
print("Grouped data as a list:", grouped_data_list)

# Step 4: Perform one-way ANOVA
anova_result = stats.f_oneway(*grouped_data_list)
print("F-statistic:", anova_result.statistic)
print("p-value:", anova_result.pvalue)


In [7]:
data = df

# Re-encode categorical variables
# Use formula-based OLS regression
formula = (
    "LOG_FREE_TRIALS ~ COST + C(CAMPAIGN_TYPE) + C(CHANNEL) + C(ATL_OR_DR) + "
    "C(SEASON) + HOLIDAY_FLAG + LAG_3_HOLIDAY_FLAG + LEAD_3_HOLIDAY_FLAG"
)
ols_model_formula = smf.ols(formula=formula, data=data).fit()

# Generate the regression summary
ols_summary_formula = ols_model_formula.summary()
ols_summary_formula


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        LOG_FREE_TRIALS   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.6402
Date:                Sun, 19 Jan 2025   Prob (F-statistic):              0.919
Time:                        21:51:37   Log-Likelihood:                -19990.
No. Observations:               14108   AIC:                         4.003e+04
Df Residuals:                   14081   BIC:                         4.024e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               10.2073      0.042    240.347      0.000      10.124      10.291
C(CAMPAIGN_TYPE)[T.Launch]               0.0099      0.045      0.221      0.825      -0.078       0.097
C(CAMPAIGN_TYPE)[T.Title]               -0.0071      0.019     -0.381      0.703      -0.043       0.029
C(CHANNEL)[T.bvod]                       0.0080      0.053      0.152      0.879      -0.095       0.111
C(CHANNEL)[T.cinema]                     0.0093      0.056      0.167      0.867      -0.100       0.118
C(CHANNEL)[T.ctv]                        0.0154      0.062      0.248      0.804      -0.106       0.137
C(CHANNEL)[T.digital audio]              0.0115      0.061      0.189      0.850      -0.108       0.131
C(CHANNEL)[T.display]                    0.0644      0.041      1.564      0.118      -0.016       0.145
C(CHANNEL)[T.olv]                        0.0112      0.061      0.182      0.855      -0.109       0.131
C(CHANNEL)[T.online display]             0.0623      0.053      1.168      0.243      -0.042       0.167
C(CHANNEL)[T.ooh]                        0.0542      0.058      0.940      0.347      -0.059       0.167
C(CHANNEL)[T.ott]                        0.0063      0.043      0.147      0.883      -0.077       0.090
C(CHANNEL)[T.paid social]                0.0134      0.037      0.364      0.716      -0.059       0.086
C(CHANNEL)[T.partnership]                0.0752      0.058      1.291      0.197      -0.039       0.189
C(CHANNEL)[T.pmax]                       0.0483      0.082      0.591      0.554      -0.112       0.208
C(CHANNEL)[T.print]                      0.1095      0.061      1.804      0.071      -0.009       0.229
C(CHANNEL)[T.radio]                      0.0988      0.064      1.542      0.123      -0.027       0.224
C(CHANNEL)[T.tv]                         0.0361      0.060      0.599      0.549      -0.082       0.154
C(CHANNEL)[T.youtube]                    0.0340      0.039      0.866      0.386      -0.043       0.111
C(ATL_OR_DR)[T.DR - Direct Response]     0.0156      0.026      0.610      0.542      -0.034       0.066
C(SEASON)[T.Spring]                     -0.0081      0.026     -0.309      0.758      -0.059       0.043
C(SEASON)[T.Summer]                     -0.0366      0.024     -1.524      0.128      -0.084       0.010
C(SEASON)[T.Winter]                     -0.0127      0.023     -0.560      0.576      -0.057       0.032
HOLIDAY_FLAG[T.True]                    -0.0305      0.044     -0.687      0.492      -0.117       0.056
LAG_3_HOLIDAY_FLAG[T.True]               0.0187      0.028      0.665      0.506      -0.037       0.074
LEAD_3_HOLIDAY_FLAG[T.True]             -0.0204      0.028     -0.729      0.466      -0.075       0.035
COST                                  

In [8]:
# # Define a function to plot boxplots and detect outliers based on IQR
# def plot_outliers(column, ax):
#     ax.boxplot(data[column], vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
#     ax.set_title(f'Boxplot of {column}')
#     ax.set_xlabel(column)

# # Create boxplots for COST and FREE_TRIALS
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# plot_outliers('COST', axes[0])
# plot_outliers('FREE_TRIALS', axes[1])
# plt.tight_layout()
# plt.show()

# Calculate outlier thresholds using IQR
def calculate_outliers(column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return lower_bound, upper_bound, outliers

# Remove outliers based on IQR thresholds for LOG_COST and LOG_FREE_TRIALS
log_cost_bounds = calculate_outliers('LOG_COST')
log_free_trials_bounds = calculate_outliers('LOG_FREE_TRIALS')

# Filter data to exclude outliers in LOG_COST and LOG_FREE_TRIALS
filtered_log_data = data[
    (data['LOG_COST'] >= log_cost_bounds[0]) & (data['LOG_COST'] <= log_cost_bounds[1]) &
    (data['LOG_FREE_TRIALS'] >= log_free_trials_bounds[0]) & (data['LOG_FREE_TRIALS'] <= log_free_trials_bounds[1])
]

# Display the size of the dataset before and after outlier removal based on log variables
log_filtered_size = filtered_log_data.shape[0]

log_filtered_size


13888

In [9]:
filtered_log_data.to_csv('../../data/filtered_data_fe.csv', index=False)

In [10]:
data = filtered_log_data

# Re-encode categorical variables
# Use formula-based OLS regression
formula = (
    "LOG_FREE_TRIALS ~ COST + C(CAMPAIGN_TYPE) + C(CHANNEL) + C(ATL_OR_DR) + "
    "C(SEASON) + HOLIDAY_FLAG + LAG_3_HOLIDAY_FLAG + LEAD_3_HOLIDAY_FLAG"
)
ols_model_formula = smf.ols(formula=formula, data=data).fit()

# Generate the regression summary
ols_summary_formula = ols_model_formula.summary()
ols_summary_formula


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        LOG_FREE_TRIALS   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.6350
Date:                Sun, 19 Jan 2025   Prob (F-statistic):              0.923
Time:                        21:52:54   Log-Likelihood:                -19305.
No. Observations:               13888   AIC:                         3.866e+04
Df Residuals:                   13861   BIC:                         3.887e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               10.1965      0.042    244.235      0.000      10.115      10.278
C(CAMPAIGN_TYPE)[T.Launch]              -0.0079      0.044     -0.180      0.857      -0.094       0.078
C(CAMPAIGN_TYPE)[T.Title]               -0.0036      0.018     -0.198      0.843      -0.039       0.032
C(CHANNEL)[T.bvod]                       0.0162      0.052      0.314      0.754      -0.085       0.117
C(CHANNEL)[T.cinema]                     0.0242      0.055      0.443      0.657      -0.083       0.131
C(CHANNEL)[T.ctv]                        0.0200      0.061      0.329      0.742      -0.099       0.139
C(CHANNEL)[T.digital audio]              0.0232      0.060      0.389      0.698      -0.094       0.140
C(CHANNEL)[T.display]                    0.0736      0.040      1.821      0.069      -0.006       0.153
C(CHANNEL)[T.olv]                        0.0281      0.060      0.469      0.639      -0.090       0.146
C(CHANNEL)[T.online display]             0.0900      0.052      1.718      0.086      -0.013       0.193
C(CHANNEL)[T.ooh]                        0.0572      0.056      1.014      0.311      -0.053       0.168
C(CHANNEL)[T.ott]                        0.0091      0.042      0.219      0.827      -0.073       0.091
C(CHANNEL)[T.paid social]                0.0210      0.036      0.580      0.562      -0.050       0.092
C(CHANNEL)[T.partnership]                0.0879      0.057      1.539      0.124      -0.024       0.200
C(CHANNEL)[T.pmax]                       0.0576      0.080      0.720      0.472      -0.099       0.214
C(CHANNEL)[T.print]                      0.1146      0.060      1.918      0.055      -0.003       0.232
C(CHANNEL)[T.radio]                      0.1019      0.063      1.621      0.105      -0.021       0.225
C(CHANNEL)[T.tv]                         0.0604      0.059      1.022      0.307      -0.055       0.176
C(CHANNEL)[T.youtube]                    0.0455      0.039      1.182      0.237      -0.030       0.121
C(ATL_OR_DR)[T.DR - Direct Response]     0.0126      0.025      0.503      0.615      -0.037       0.062
C(SEASON)[T.Spring]                     -0.0143      0.026     -0.560      0.576      -0.065       0.036
C(SEASON)[T.Summer]                     -0.0224      0.024     -0.950      0.342      -0.069       0.024
C(SEASON)[T.Winter]                     -0.0149      0.022     -0.670      0.503      -0.058       0.029
HOLIDAY_FLAG[T.True]                    -0.0294      0.044     -0.676      0.499      -0.115       0.056
LAG_3_HOLIDAY_FLAG[T.True]               0.0138      0.028      0.499      0.618      -0.040       0.068
LEAD_3_HOLIDAY_FLAG[T.True]             -0.0153      0.028     -0.553      0.580      -0.069       0.039
COST                                  

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns to standardize
numerical_features = ['LOG_COST', 'LOG_COST_SQ']
scaler = StandardScaler()

# Standardize numerical features
filtered_log_data_encoded[numerical_features] = scaler.fit_transform(
    filtered_log_data_encoded[numerical_features]
)

# Re-run OLS regression with standardized features
X_standardized = filtered_log_data_encoded[['LOG_COST', 'LOG_COST_SQ'] +
                                           [col for col in filtered_log_data_encoded.columns
                                            if col.startswith(('ATL_OR_DR_', 'CAMPAIGN_TYPE_', 'CHANNEL_'))]]
X_standardized = sm.add_constant(X_standardized)  # Add constant for OLS

y_standardized = filtered_log_data_encoded['LOG_FREE_TRIALS']

# Fit OLS regression with standardized features
ols_standardized_model = sm.OLS(y_standardized, X_standardized).fit()

# Display the updated summary
ols_standardized_summary = ols_standardized_model.summary()
ols_standardized_summary